In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Configurar el generador de números aleatorios para reproducibilidad
np.random.seed(42)

# Generar fechas para un año de mediciones diarias
start_date = datetime(2023, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(365)]

# Generar datos
n_samples = len(dates) * 5  # 5 mediciones por día

# Profundidad en metros (0 a 1000 metros)
depth = np.random.uniform(0, 1000, n_samples)

# Temperatura en grados Celsius
# Asumimos una relación lineal con algo de ruido
# Temperatura superficial media: 25°C
# Tasa de disminución: aproximadamente 0.65°C cada 100 metros
base_temperature = 25 - 0.0065 * depth

# Añadir variación estacional (más cálido en verano, más frío en invierno)
day_of_year = np.array([(date.timetuple().tm_yday - 1) / 365 * 2 * np.pi for date in dates for _ in range(5)])
seasonal_variation = 2 * np.sin(day_of_year)  # ±2°C variación estacional

# Añadir variación aleatoria
random_variation = np.random.normal(0, 0.5, n_samples)

temperature = base_temperature + seasonal_variation + random_variation

# Generar datos de salinidad (en PSU - Practical Salinity Unit)
# La salinidad tiende a aumentar ligeramente con la profundidad
base_salinity = 35 + 0.001 * depth
salinity = base_salinity + np.random.normal(0, 0.1, n_samples)

# Generar datos de oxígeno disuelto (en mg/L)
# El oxígeno disuelto tiende a disminuir con la profundidad y la temperatura
oxygen = 8 - 0.005 * depth - 0.1 * temperature + np.random.normal(0, 0.2, n_samples)
oxygen = np.clip(oxygen, 0, 10)  # Asegurar que el oxígeno no sea negativo y no exceda 10 mg/L

# Crear DataFrame
df = pd.DataFrame({
    'fecha': dates * 5,
    'profundidad': depth,
    'temperatura': temperature,
    'salinidad': salinity,
    'oxigeno_disuelto': oxygen
})

# Ordenar el DataFrame por fecha y profundidad
df = df.sort_values(['fecha', 'profundidad'])

# Guardar a CSV
csv_path = 'temperatura_oceano.csv'
df.to_csv(csv_path, index=False)

print(f"Archivo '{csv_path}' generado exitosamente.")

# Mostrar las primeras filas y un resumen estadístico
print("\nPrimeras filas del conjunto de datos:")
print(df.head())

print("\nResumen estadístico:")
print(df.describe())

print("\nInformación del DataFrame:")
print(df.info())

Archivo 'temperatura_oceano.csv' generado exitosamente.

Primeras filas del conjunto de datos:
          fecha  profundidad  temperatura  salinidad  oxigeno_disuelto
730  2023-01-01   164.265798    25.277250  35.199887          4.649985
1095 2023-01-01   176.528032    23.048905  35.241471          4.457954
0    2023-01-01   374.540119    22.273810  35.389784          4.087915
365  2023-01-01   386.102638    24.407527  35.439212          3.784329
1460 2023-01-01   393.864406    20.565936  35.470789          3.793302

Resumen estadístico:
       profundidad  temperatura    salinidad  oxigeno_disuelto
count  1825.000000  1825.000000  1825.000000       1825.000000
mean    497.175091    21.760324    35.496726          3.326879
std     293.186572     2.444769     0.307196          1.301074
min       4.632023    15.743067    34.806663          0.649554
25%     236.870559    19.973893    35.247117          2.241764
50%     503.136259    21.789566    35.505528          3.316029
75%     749.9604